#### Build

In [1]:
# Upload the Github Zipfile
# Unzip the Github Zipfile
# Create python 3.8             conda create --name cse297 python=3.8.10
# Activate the environment      source activate cse297
# Within env install ipyk       pip install ipykernel
# Register kernel w/ jupy       python -m ipykernel install --user --name=capstone
# Change to package dir         cd CybORG-Competitive/CybORG/
# Install packages within env   pip install -e .
# Select 'capstone' kernel
# Confirm python version is 3.8.10
import sys
print(sys.version)

3.8.19 (default, Mar 20 2024, 19:55:45) [MSC v.1916 64 bit (AMD64)]


#### Import

In [2]:
from environments import build_blue_agent, build_red_agent, sample, get_timesteps, get_algorithm_select
from environments import build_cardiff_agent, sample_against_cardiff

import ray
import os, sys, shutil, time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import sleep

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
import logging
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
logger = logging.getLogger(__name__)

In [3]:
### Testing code to train red agaent against cardiff blue agent
red = build_red_agent(fresh=True,dedicated=True,vs_cardiff=True)
selected_timestep = get_timesteps()
selected_algorithm = get_algorithm_select()

# load agent
path_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/red_dedicated_pool/dedicated_red_0/checkpoint_path", "r")
red_restore_path = path_file.read()
path_file.close()
red.restore(red_restore_path)

# dedicated training
b = 4
for i in range(b):
    start = time.time()
    result = red.train()
    end = time.time()
    elapsed_time = end-start
    # Algorithm specific score retrieval
    red_score = None
    if(selected_algorithm == "ppo" or selected_algorithm == "dqn"):
        red_score = result["sampler_results"]["episode_reward_mean"]
    else:
        red_score = result["episode_reward_mean"]
    entropy = vf_loss = 0
    if(selected_algorithm != "dqn"):
        entropy = result['info']['learner']['default_policy']['learner_stats']['entropy']
        vf_loss = result['info']['learner']['default_policy']['learner_stats']['vf_loss']
    print(f"Batch {i} -- Red Score: {red_score:0.2f}    Entropy: {entropy:0.2f}    VF Loss: {vf_loss:0.2f}", end="    ")
    print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

selecting red ppo config


(RolloutWorker pid=3736) C:\Users\takys\miniconda3\envs\cse297\lib\site-packages\ray\rllib\algorithms\algorithm.py:28: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
(RolloutWorker pid=3736)   import pkg_resources
(RolloutWorker pid=3736) C:\Users\takys\miniconda3\envs\cse297\lib\site-packages\pkg_resources\__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
(RolloutWorker pid=3736) Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
(RolloutWorker pid=3736)   declare_namespace(pkg)
(RolloutWorker pid=3736) C:\Users\takys\miniconda3\envs\cse297\lib\site-packages\pkg_resources\__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
(RolloutWorker pid=3736) Imp

(RolloutWorker pid=3736) should be here!
(RolloutWorker pid=21700) should be here!
(RolloutWorker pid=2396) should be here!
(RolloutWorker pid=9452) should be here!
./policies/ppo/30/red_dedicated_pool/dedicated_red_0\checkpoint_000000
(RolloutWorker pid=3736) MainAgent.py - get_action() - agent chose action:  51
(RolloutWorker pid=3736) wrapper.py - step()
(RolloutWorker pid=3736) cardiff is  True
(RolloutWorker pid=3736) red_step is:  PrivilegeEscalate User3
(RolloutWorker pid=3736) blu_step is:  DecoyFemitter User2
(RolloutWorker pid=3736) MainAgent.py - get_action() - agent chose action:  116
(RolloutWorker pid=3736) wrapper.py - step()
(RolloutWorker pid=3736) cardiff is  True
(RolloutWorker pid=3736) red_step is:  ExploitRemoteService 10.0.67.193
(RolloutWorker pid=3736) blu_step is:  DecoyTomcat User2
(RolloutWorker pid=3736) MainAgent.py - get_action() - agent chose action:  55
(RolloutWorker pid=3736) wrapper.py - step()
(RolloutWorker pid=3736) cardiff is  True
(RolloutWorker

In [ ]:
### Testing code for building a cardiff blue agent ( to be moved down to the correct cell later below!)
blue_cardiff = build_cardiff_agent()
print("blu_cardiff is: ",blue_cardiff)
blue = build_blue_agent(fresh=True, opponent=True)
print("blu is: ", blue)

# Retrieve the timestep and the algorithm selected for training
selected_timestep = get_timesteps()
selected_algorithm = get_algorithm_select()
red = build_red_agent(fresh=False)

# Load the 'optimal' red agent policy
red_optimal_filepath = None
with open(f"./policies/{selected_algorithm}/{selected_timestep}/competitive_red_policy", "r") as red_file:
    red_optimal_filepath = red_file.read()
red.restore(red_optimal_filepath)
print("starting sample game...")

# Sample game
sample_against_cardiff(red,blue_cardiff,games=1,verbose=True)

#### Verify GPU

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

#### Train Competitive Red Agent

In [ ]:
# Retrieve the timestep and the algorithm selected for training
selected_timestep = get_timesteps()
selected_algorithm = get_algorithm_select()

# Select which generation we want to start training from
# If we want to train from scratch set to 1 (default value is 1)
# If we want to train from the latest generation, set to latest generation in the competitive policy pool
start_from_generation = 1

# Load rewards if training from a checkpointed generation
blue_scores = []
red_scores = []

# Total number of generations create for red agent
generations = 25

# Error checking: Update either the 'starting_from_generation' or 'total_generations' parameter
if(start_from_generation > generations):
    raise ValueError("Starting generation and Total Generation incompatible")

# Number of batches without improvement before ending training
tolerance = 3

# Create Initial Policies
ray.init(ignore_reinit_error=True, log_to_driver=False)
blue = build_blue_agent(fresh=True, opponent=True)
print("Pass 1")
red = build_red_agent(fresh=True)
print("Pass 2")

# Convert float string into a float
# Useful if loading rewards from certain generation
blue_scores = [float(item) for item in blue_scores]
red_scores = [float(item) for item in red_scores]

print()
print("+--------------------------------+")
print("| Red Competitive Training Start |")
print("+--------------------------------+")
print()

for g in range(start_from_generation, generations+1):

    # Time how long each generation takes
    g_time = time.time()

    if (g < 10):
        dashes = 14
    elif (g < 100):
        dashes = 15
    else:
        dashes = 16
    print('+'+'-'*dashes+'+')            
    print(f"| Generation {g} |")
    print('+'+'-'*dashes+'+')
    print()

    red.restore(f"./policies/{selected_algorithm}/{selected_timestep}/red_competitive_pool/competitive_red_0/checkpoint_000000")

    b = 0
    red_max = 0
    tol = tolerance
    b_time = time.time()
    while True:
        b += 1
        start = time.time()
        result = red.train()
        # print(result)
        end = time.time()
        elapsed_time = end-start
        # Algorithm specific score retrieval
        red_score = None
        if(selected_algorithm == "ppo" or selected_algorithm == "dqn"):
            red_score = result["sampler_results"]["episode_reward_mean"]
        else:
            red_score = result["episode_reward_mean"]
        entropy = vf_loss = 0
        if(selected_algorithm != "dqn"):
            entropy = result['info']['learner']['default_policy']['learner_stats']['entropy']
            vf_loss = result['info']['learner']['default_policy']['learner_stats']['vf_loss']
        print(f"Batch {b} -- Red Score: {red_score:0.2f}    Entropy: {entropy:0.2f}    VF Loss: {vf_loss:0.2f}", end="    ")
        print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        if b > 1:
            if (red_score > red_max):
                red_max = red_score
                tol = tolerance
                checkpoint_path = red.save(checkpoint_dir=f"./policies/{selected_algorithm}/{selected_timestep}/red_competitive_pool/competitive_red_{g}")
                path_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/red_competitive_pool/competitive_red_{g}/checkpoint_path", "w")
                path_file.write(checkpoint_path)
                path_file.close()
            elif(tol > 1):
                tol -= 1
             # when agent is no longer improving, break and save the new best-response agent
            else:
                red_scores.append(red_max)
                red.restore(checkpoint_path)
                print('Batch time:', time.strftime("%H:%M:%S", time.gmtime(time.time()-b_time)))
                print(checkpoint_path)
                break

    pool_size = g
    pool_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/red_competitive_pool/pool_size", "w")
    pool_file.write(str(pool_size))
    pool_file.close()
    print()

    blue.restore(f"./policies/{selected_algorithm}/{selected_timestep}/blue_opponent_pool/opponent_blue_0/checkpoint_000000")

    b = 0 # b tracks the batches of training completed
    blue_min = float('inf')
    tol = tolerance
    b_time = time.time()
    while True:
        b += 1
        start = time.time()
        result = blue.train()
        end = time.time()
        elapsed_time = end-start
        
        # Score retrieval based on algorithm
        blue_score = None
        if(selected_algorithm == "ppo" or selected_algorithm == "dqn"):
            blue_score = -result["sampler_results"]["episode_reward_mean"]
        else:
            blue_score = -result["episode_reward_mean"]
        entropy = vf_loss = 0
        if(selected_algorithm != "dqn"):
            entropy = result['info']['learner']['default_policy']['learner_stats']['entropy']
            vf_loss = result['info']['learner']['default_policy']['learner_stats']['vf_loss']
        print(f"Batch {b} -- Blue Score: {blue_score:0.2f}    Entropy: {entropy:0.2f}    VF Loss: {vf_loss:0.2f}", end="    ")
        print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        if b > 1:
            if (blue_score < blue_min):
                blue_min = blue_score
                tol = tolerance
                checkpoint_path = blue.save(checkpoint_dir=f"./policies/{selected_algorithm}/{selected_timestep}/blue_opponent_pool/opponent_blue_{g}")
                path_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/blue_opponent_pool/opponent_blue_{g}/checkpoint_path", "w")
                path_file.write(checkpoint_path)
                path_file.close()
            elif(tol > 1):
                tol -= 1
            # when agent is no longer improving, break and save the new competitive agent
            else:
                blue_scores.append(blue_min)
                blue.restore(checkpoint_path)
                print('Batch time:', time.strftime("%H:%M:%S", time.gmtime(time.time()-b_time)))
                print(checkpoint_path)
                break

    pool_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/blue_opponent_pool/pool_size", "w")
    pool_file.write(str(pool_size))
    pool_file.close()
    print()

    print(f'Blue Scores so far {["%.2f" % i for i in blue_scores]}')
    print(f'Red Scores so far {["%.2f" % i for i in red_scores]}')
    print("Total time: ",time.strftime("%H:%M:%S", time.gmtime(time.time()-g_time)))
    print()
    
    # print(f'-------- Sample Game for Generation {g} --------')
    # sample(red, blue, verbose=True, show_policy=True)
    # print()

#### Evaluate Red MinMax Performance

In [ ]:
# Set the sample games (default is 50)
sample_games = 1

r_min = [float('inf')]*(generations+1) # red agent minimum scores
r_min_op = [0]*(generations+1) # id of blue opponent that got max score
r_best_score = 0
r_best_id = 0

# evaluate existing pool of agents
print('Evaluating Agents...')
g_start = time.time()

# iteration through red agents
for r in range(generations,0,-1):

    # If the generation exists
    if(os.path.exists(f"./policies/{selected_algorithm}/{selected_timestep}/red_competitive_pool/competitive_red_{r}/checkpoint_path")):
        start = time.time()
        path_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/red_competitive_pool/competitive_red_{r}/checkpoint_path", "r")
        red_restore_path = path_file.read()
        path_file.close()
        red.restore(red_restore_path)
    
        # iterate through blue opponents
        for b in range(generations,0,-1):

            # If the generation exists
            if(os.path.exists(f"./policies/{selected_algorithm}/{selected_timestep}/blue_opponent_pool/opponent_blue_{b}/checkpoint_path")):
                path_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/blue_opponent_pool/opponent_blue_{b}/checkpoint_path", "r")
                blue_restore_path = path_file.read()
                path_file.close()
                blue.restore(blue_restore_path)
                score = sample(red, blue, games=sample_games)
                if score < r_min[r]:
                    r_min[r] = score
                    r_min_op[r] = b
                    
        print(f'Red Agent {r} expects a minimum of {r_min[r]:0.2f} points, against Blue Opponent {r_min_op[r]}.', end="\t")
        print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(time.time()-start)))
        if r_min[r] > r_best_score:
            r_best_score = r_min[r]
            r_best_id = r
print()
print(f'Top performing Red Agent is generation {r_best_id}')
print("Total Execution Time: ", time.strftime("%H:%M:%S", time.gmtime(time.time()-g_start)))

path_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/red_competitive_pool/competitive_red_{r_best_id}/checkpoint_path", "r")
red_competitive_path = path_file.read()
path_file.close()
path_file = open(f"./policies/{selected_algorithm}/{selected_timestep}/competitive_red_policy", "w")
path_file.write(red_competitive_path)
path_file.close()

ray.shutdown()

#### Observe Exploitability of each Red Policy

In [ ]:
r_exp = [] # exploitability of each Red Policy
for r in r_min[1:]:
    r_exp.append(r_best_score-r)
print(r_exp)

#### Plot Red Training Scores

In [ ]:
id_plot = [id for id in range(1,len(red_scores)+1)]
data_plot = pd.DataFrame({"Generation":id_plot, "Training Score":red_scores, "Blue Scores":blue_scores})
plt.figure()
sns.lineplot(x = "Generation", y = "Training Score", data=data_plot, color='red', label="Red Agent Training Score")
sns.lineplot(x = "Generation", y = "Blue Scores", data=data_plot, color='blue', label="Blue Opponent Score")
plt.show()

# Minmax Evaluation
id_plot = [id for id in range(len(red_scores))]
data_plot = pd.DataFrame({"Generation":id_plot, "Min Expected Score":r_min[1:]})
plt.figure()
sns.regplot(x = "Generation", y = "Min Expected Score", data=data_plot, color='red', scatter_kws={'s':5}, label="Red Policy Min Expected Score")
plt.legend()
plt.show()

# Exploitability
id_plot = [id for id in range(len(red_scores))]
data_plot = pd.DataFrame({"Generation":id_plot, "Exploitability":r_exp})
plt.figure()
sns.regplot(x = "Generation", y = "Exploitability", data=data_plot, color='red', scatter_kws={'s':5}, label="RedPolicy Exploitability")
plt.legend()
plt.show()

#### Plot Red Training Scores (First 5 points dropped)

In [ ]:
# Scores During Training
id_plot = [id for id in range(1,len(red_scores)+1)]
data_plot = pd.DataFrame({"Generation":id_plot[5:], "Training Score":red_scores[5:], "Blue Scores":blue_scores[5:]})
plt.figure()
sns.lineplot(x = "Generation", y = "Training Score", data=data_plot, color='red', label="Red Agent Training Score")
sns.lineplot(x = "Generation", y = "Blue Scores", data=data_plot, color='blue', label="Blue Opponent Score")
plt.show()

# Minmax Evaluation
id_plot = [id for id in range(len(red_scores))]
data_plot = pd.DataFrame({"Generation":id_plot[5:], "Min Expected Score":r_min[6:]})
plt.figure()
sns.regplot(x = "Generation", y = "Min Expected Score", data=data_plot, color='red', scatter_kws={'s':5}, label="Red Policy Min Expected Score")
plt.legend()
plt.show()

# Exploitability
id_plot = [id for id in range(len(red_scores))]
data_plot = pd.DataFrame({"Generation":id_plot[5:], "Exploitability":r_exp[5:]})
plt.figure()
sns.regplot(x = "Generation", y = "Exploitability", data=data_plot, color='red', scatter_kws={'s':5}, label="Red Policy Exploitability")
plt.legend()
plt.show()

#### Cardiff Blue Agent vs Competitive Red PPO Agent

In [ ]:
# Red Agent - Restore the best checkpointed path
# Blu Agent - Restore Cardiff blue agent

#### Cardiff Blue Agent & Dedicated Red PPO Agent Training